# W&B → Cloud Deploy Lab — Step-by-step Notebook

**Purpose:** Teach students how to train a simple model, log it to Weights & Biases (W&B) as an artifact, create a minimal inference service (FastAPI) which downloads the model artifact at startup, containerize the service, and deploy it to Google Cloud Run. The notebook is written so the training + artifact steps can be executed in Colab or a Jupyter server. Docker & cloud deploy steps are runnable on local Ubuntu lab machines.

**What you will produce:**

- A logged W&B model artifact (`iris-rf`) in project `classroom-deploy`.
- A FastAPI inference app that downloads the artifact and serves `/predict`.
- A Dockerfile to containerize the app.
- Cloud Run deployment instructions (or substitute AWS/Azure as desired).

---


## Prerequisites (before starting)

- **W&B account & API key**: Make sure students have W&B accounts (academic) and their `WANDB_API_KEY`.
- **Python**: Colab or Python 3.8+ environment.
- **For Docker & deploy**: Local Ubuntu machines with Docker and Google Cloud SDK installed.
- **Optional**: GitHub account (for CI/CD exercise).

**Notes:**
- Colab: You can run the training and artifact steps in Colab, but building Docker images and deploying with `gcloud` must be done on local machines.



## 1) Install Python dependencies

Run this cell in Colab or your local notebook to install the required packages for the lab (training, logging, and local testing).

In [1]:
!pip install --quiet wandb scikit-learn joblib numpy
print('Installed core packages (wandb, scikit-learn, joblib, numpy)')

Installed core packages (wandb, scikit-learn, joblib, numpy)


### 1.1 Set your W&B API key

There are two ways:

- **Option A — Colab / notebook only (ephemeral)**:

```python
import os
os.environ['WANDB_API_KEY'] = 'your_wandb_api_key_here'
```

- **Option B — Use `wandb login` interactively (recommended for Colab):**

```bash
!wandb login
```

- **Option C — Local Ubuntu**:

```bash
export WANDB_API_KEY='your_wandb_api_key_here'
```

Make sure `WANDB_API_KEY` is set before running W&B code.



In [2]:
import os
os.environ['WANDB_API_KEY'] = 'c1e6a0d4272084f5d33850fd498723e1ccf4f403'

In [3]:
# Quick check for WANDB_API_KEY
import os
if 'WANDB_API_KEY' in os.environ:
    print('WANDB_API_KEY is set (will be used by wandb.login()).')
else:
    print('WANDB_API_KEY not set. Run `os.environ[...] = '"your_key"'` in Colab or `!wandb login`.')

WANDB_API_KEY is set (will be used by wandb.login()).


## 2) Login to W&B from the notebook

This will attempt to use `WANDB_API_KEY` from the environment. If not set, `wandb.login()` may prompt you interactively (Colab).

In [4]:
import wandb
try:
    wandb.login()
    print('wandb.login() successful (or WANDB_API_KEY used).')
except Exception as e:
    print('wandb.login() failed — set WANDB_API_KEY or run `!wandb login` interactively.\n', e)

wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: sairohith (ir2023) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb.login() successful (or WANDB_API_KEY used).


In [5]:
# !wandb login --relogin

## 3) Step 1 — Train a model and log it to W&B as an artifact

This cell trains a small RandomForest on the Iris dataset, saves `model.pkl`, and logs it to W&B in project `classroom-deploy` with artifact name `iris-rf`.

**Run this cell**.

In [6]:
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
import joblib
import wandb

print('Training model...')

run = wandb.init(project='classroom-deploy', job_type='train')
X, y = load_iris(return_X_y=True)
clf = RandomForestClassifier(n_estimators=50, random_state=42)
clf.fit(X, y)
acc = clf.score(X, y)
print(f'Training accuracy: {acc:.4f}')

model_path = 'model.pkl'
joblib.dump(clf, model_path)

artifact = wandb.Artifact(name='iris-rf', type='model', metadata={'accuracy': acc})
artifact.add_file(model_path)
run.log_artifact(artifact)
print('Logged artifact: iris-rf')
run.finish()

Training model...


Training accuracy: 1.0000
Logged artifact: iris-rf


### 3.1 Verify your artifact

- Open the W&B web UI (https://wandb.ai) → your account → project `classroom-deploy` and you should see the run and the artifact.

- Optional: try to list artifacts via the API. If you encounter permission/enterprise differences, rely on the web UI.


In [14]:
import wandb
api = wandb.Api()
try:
    print('Recent model artifacts (project=classroom-deploy):')
    arts = api.artifacts(type_name='model', name='ir2023/classroom-deploy/iris-rf')
    for a in arts[:10]:
        try:
            print('-', a.id)
        except Exception:
            print('-', str(a))
except Exception as e:
    print('Could not list artifacts via API. Please check the W&B web UI to inspect artifacts.\n', e)

Recent model artifacts (project=classroom-deploy):
- QXJ0aWZhY3Q6MjE1NzUyOTM2Ng==
- QXJ0aWZhY3Q6MjE1NjM3Nzg3Mg==
- QXJ0aWZhY3Q6MjEzMzUwNTI4MA==
- QXJ0aWZhY3Q6MjEzMzQzNTU3Ng==
- QXJ0aWZhY3Q6MjEzMzQwOTE4MA==
- QXJ0aWZhY3Q6MjEzMzQwNTk1OQ==
- QXJ0aWZhY3Q6MjEzMzM4MDQ2NA==
- QXJ0aWZhY3Q6MjEyMjQ0NTUxMw==
- QXJ0aWZhY3Q6MjEyMjQwNzI4Ng==


## 4) Step 2 — Create a FastAPI app that downloads the model artifact at startup

Run the following cell to create `app/main.py`, `app/requirements.txt`, and a `Dockerfile`. The app expects an environment variable `WANDER_MODEL_ARTIFACT` with the artifact reference, e.g. `your-username/classroom-deploy/iris-rf:latest`. If you want to bake the model into the image, see the notes below.


In [16]:
import os
os.makedirs('app', exist_ok=True)

main_py = '''import os
import joblib
import wandb
from fastapi import FastAPI, Request
import numpy as np

app = FastAPI()
MODEL_ARTIFACT = os.environ.get('WANDER_MODEL_ARTIFACT', 'ir2023/classroom-deploy/iris-rf:v8')

# Downloads the artifact and returns a loaded model
def load_model_from_wandb(artifact_ref):
    try:
        wandb.login()
    except Exception:
        pass
    api = wandb.Api()
    artifact = api.artifact(artifact_ref)
    path = artifact.download()
    model_file = os.path.join(path, 'model.pkl')
    return joblib.load(model_file)

@app.on_event('startup')
def startup():
    global model
    model = load_model_from_wandb(MODEL_ARTIFACT)

@app.get('/')
def root():
    return {'status': 'ok', 'model_artifact': MODEL_ARTIFACT}

@app.post('/predict')
async def predict(request: Request):
    features = await request.json()
    arr = np.array(features).reshape(1, -1)
    pred = model.predict(arr)
    return {'prediction': int(pred[0])}
'''

with open('app/main.py', 'w') as f:
    f.write(main_py)

reqs = '''fastapi
uvicorn[standard]
scikit-learn
joblib
wandb
numpy
'''
with open('app/requirements.txt', 'w') as f:
    f.write(reqs)

dockerfile = '''FROM python:3.10-slim
WORKDIR /app
COPY ./requirements.txt /app/requirements.txt
RUN pip install --no-cache-dir -r /app/requirements.txt
COPY ./main.py /app/main.py
ENV PORT=8080
CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8080"]
'''
with open('app/Dockerfile', 'w') as f:
    f.write(dockerfile)

print('Wrote files to ./app:')
print(' - app/main.py')
print(' - app/requirements.txt')
print(' - app/Dockerfile')


Wrote files to ./app:
 - app/main.py
 - app/requirements.txt
 - app/Dockerfile


In [18]:
print('\n--- app/main.py ---\n')
print(open('app/main.py').read()[:1000])
print('\n(Truncated output)')


--- app/main.py ---

import os
import joblib
import wandb
from fastapi import FastAPI, Request
import numpy as np

app = FastAPI()
MODEL_ARTIFACT = os.environ.get('WANDER_MODEL_ARTIFACT', 'ir2023/classroom-deploy/iris-rf:v8')

# Downloads the artifact and returns a loaded model
def load_model_from_wandb(artifact_ref):
    try:
        wandb.login()
    except Exception:
        pass
    api = wandb.Api()
    artifact = api.artifact(artifact_ref)
    path = artifact.download()
    model_file = os.path.join(path, 'model.pkl')
    return joblib.load(model_file)

@app.on_event('startup')
def startup():
    global model
    model = load_model_from_wandb(MODEL_ARTIFACT)

@app.get('/')
def root():
    return {'status': 'ok', 'model_artifact': MODEL_ARTIFACT}

@app.post('/predict')
async def predict(request: Request):
    features = await request.json()
    arr = np.array(features).reshape(1, -1)
    pred = model.predict(arr)
    return {'prediction': int(pred[0])}


(Truncated output)


### 4.1 Quick test: Download the artifact and run inference locally (no FastAPI server needed)

This lets students verify the model artifact and inference logic inside the notebook. It downloads the artifact using the W&B API and loads `model.pkl` then runs a test prediction.


In [19]:
import joblib
import wandb

artifact_ref = 'ir2023/classroom-deploy/iris-rf:v8'  # change to your artifact ref
print('Attempting to download artifact:', artifact_ref)
try:
    api = wandb.Api()
    artifact = api.artifact(artifact_ref)
    path = artifact.download()
    print('Downloaded to', path)
    model = joblib.load(os.path.join(path, 'model.pkl'))
    import numpy as np
    sample = np.array([5.1, 3.5, 1.4, 0.2]).reshape(1, -1)
    print('Sample prediction:', model.predict(sample).tolist())
except Exception as e:
    print('Could not download/run model locally. Check WANDB_API_KEY and artifact ref.\n', e)

Attempting to download artifact: ir2023/classroom-deploy/iris-rf:v8


wandb:   1 of 1 files downloaded.  


Downloaded to c:\Users\sairo\Desktop\MLOps\mlops2025w_142201019\class\week-6\artifacts\iris-rf-v8


c:\Users\sairo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.7.1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\sairo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.7.1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Sample prediction: [0]


### 4.2 Run the FastAPI app locally (non-Docker) — good for quick testing on a development machine

From the project root (where `app/` lives), run in a terminal:

```bash
# make sure WANDB_API_KEY and WANDER_MODEL_ARTIFACT are set in the environment, example:
export WANDB_API_KEY='8e3ce4890d24a75bc4fad43f0961d5b7338f3bbf'
export WANDER_MODEL_ARTIFACT='ir2023/classroom-deploy/iris-rf:v0'

# install requirements (if running outside notebook):
pip install -r app/requirements.txt

# run server
uvicorn app.main:app --host 0.0.0.0 --port 8080
```

Then in another terminal test:

```bash
curl -X POST -H "Content-Type: application/json" --data '[5.1,3.5,1.4,0.2]' http://localhost:8080/predict
```



In [11]:
!export WANDB_API_KEY='c1e6a0d4272084f5d33850fd498723e1ccf4f403'


'export' is not recognized as an internal or external command,
operable program or batch file.


In [12]:
!export WANDER_MODEL_ARTIFACT='ir2023/classroom-deploy/iris-rf:v0'

'export' is not recognized as an internal or external command,
operable program or batch file.


In [21]:
# !curl -X POST -H "Content-Type: application/json" --data '[5.1,3.5,1.4,0.2]' http://localhost:8080/predict

import requests
import json

# URL of the API endpoint
url = "http://localhost:8080/predict"

# Data to send (same as in your curl command)
data = [5.1, 3.5, 1.4, 0.2]

# Make the POST request
response = requests.post(url, headers={"Content-Type": "application/json"}, data=json.dumps(data))

# Print the response text (or JSON if applicable)
print(response.text)


{"prediction":0}


## 5) Dockerize the app — build and run on local Ubuntu lab machines

**Build** (from project root):

```bash
cd app
docker build -t iris-wandb:local .
```

**Run** (pass secrets via env vars):

```bash
docker run -e WANDB_API_KEY="$WANDB_API_KEY" \
  -e WANDER_MODEL_ARTIFACT="your-username/classroom-deploy/iris-rf:latest" \
  -p 8080:8080 iris-wandb:local
```

**Test** as before with curl to `/predict`.

**Baking model into image (optional)**: For faster startup and offline runs, download the artifact locally and `COPY` the `model.pkl` into the image during build. That requires a local download step prior to `docker build`.


## 6) Deploy to Google Cloud Run

Prerequisite: `gcloud` configured and authenticated, billing enabled for the project.

Commands (run locally):

```bash
# set project
gcloud config set project YOUR_GCP_PROJECT

# build and push using Cloud Build
gcloud builds submit --tag gcr.io/$GOOGLE_CLOUD_PROJECT/iris-wandb

# deploy
gcloud run deploy iris-wandb \
  --image gcr.io/$GOOGLE_CLOUD_PROJECT/iris-wandb \
  --platform managed \
  --region us-central1 \
  --allow-unauthenticated \
  --set-env-vars=WANDB_API_KEY=$WANDB_API_KEY,WANDER_MODEL_ARTIFACT=your-username/classroom-deploy/iris-rf:latest
```

After this, Cloud Run will provide a public URL you can call with the same `/predict` POST body.


## 7) Optional: GitHub Actions for CI/CD (auto-deploy on push)

Create `.github/workflows/ci-cd.yml` with the following content (example for Cloud Run deployment):

```yaml
name: CI/CD
on:
  push:
    branches: [ main ]
jobs:
  build-deploy:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      - uses: google-github-actions/setup-gcloud@v1
        with:
          project_id: ${{ secrets.GCP_PROJECT }}
          service_account_key: ${{ secrets.GCP_SA_KEY }}
      - name: Build & push
        run: |
          gcloud builds submit --tag gcr.io/$GCP_PROJECT/iris-wandb
      - name: Deploy to Cloud Run
        run: |
          gcloud run deploy iris-wandb \
            --image gcr.io/$GCP_PROJECT/iris-wandb \
            --platform managed \
            --region us-central1 \
            --allow-unauthenticated \
            --set-env-vars=WANDB_API_KEY=${{ secrets.WANDB_API_KEY }},WANDER_MODEL_ARTIFACT=your-username/classroom-deploy/iris-rf:latest
```

Use GitHub Secrets for `GCP_SA_KEY` and `WANDB_API_KEY`.


## 8) Troubleshooting & common issues

- **W&B authentication errors**: Ensure `WANDB_API_KEY` is set or run `wandb login`.
- **Artifact not found**: Check artifact path format: `username/project/artifact-name:version` or `username/project/artifact-name:latest`.
- **Docker permissions**: If `docker build` fails due to permissions, ensure your user can run Docker or use `sudo`.
- **Startup latency**: Downloading the artifact at container startup adds latency — consider baking the model into the image for production.



## Instructor notes & grading rubric

Suggested rubric:
- Model logged as W&B Artifact: 10 pts
- Local FastAPI app runs and predicts (no Docker): 20 pts
- Docker image build successful: 20 pts
- Cloud Run (or cloud) deployment functional: 30 pts
- README / Documentation clarity: 20 pts

Instructor tip: Pre-create a W&B project and share artifact refs to reduce setup noise in class.
